## ZAP Sessions
ZAP allows the user to store a session for a crawled application, reload that into context and perform scan operations on the session. However, there are few things to remember here.
1. You can only use a session file once. Once the session is used for scan operations, it is rendered "tainted" which means that it has scan results, etc. 
2. So, in order to use a session file to perform "parameterization", its advisable to store the untainted version of the session file in a location, copy it over to another location and then load the copied session file in ZAP for scanning, etc. 

Here we have a session file called `api.session` in the `orig_session` directory. We are going to copy the `api.session` over to the `temp_session` directory and perform ZAP Scanning Operations on it, programmatically. We will then use the copied session file at ZAP and perform, scanning, etc. 

### Copying the session file to `temp_session` folder

In [15]:
import shutil
import os
import glob

src = 'orig_session/'
dst = 'temp_session/'
for filename in glob.glob(os.path.join(src, '*.*')):
    shutil.copy(filename, dst)
## check if file exists in temp session directory
if os.path.isfile('temp_session/api.session'):
    print("File has been copied successfully")
else:
    print("File has not been found in temp session directory")

File has been copied successfully


### Start ZAP and initialize API

In [2]:
# we will be using python's subprocess to start ZAP in GUI and headless modes

import subprocess
import os
from IPython.display import display

#GUI ZAP
base_path = '/Applications/OWASP_ZAP.app/Contents/Java/'
gui_command = base_path + 'zap.sh -config api.disablekey=true -port 8090'
# you can use the config param to specify set specific configurations you need when you launch the CLI.
# In this case, I am (actually don't need to) starting ZAP with the API Key disabled and listening port 8090

headless_command = base_path + 'zap.sh -daemon -config api.disablekey=true -port 8090'
#by specifying 'daemon' in the CLI, ZAP starts in Headless mode

zap_process = subprocess.Popen(gui_command.split(' '), stdout = open(os.devnull, 'w'))

### Initialize ZAP API

In [16]:
from zapv2 import ZAPv2 as ZAP #import ZAP library
import time

zap = ZAP(proxies = {'http': 'http://localhost:8090', 'https': 'http://localhost:8090'})
#setting the local ZAP instance that is open on your local system

### Load Session

In [17]:
session_file = os.path.join(os.getcwd(), 'temp_session/api.session')
print(session_file)
zap.core.load_session(session_file)

/Users/abhaybhargav/Documents/Code/Python/zap_mini_workshop/temp_session/api.session


'OK'

### Run Active Scan on Host

In [18]:
# using ZAP's ascan object to start scanning, with the "Light" Policy. If you don't specify the policy
# ZAP Automatically uses the "Default" policy

target_url = 'http://localhost:5050/'
active_scan_id = zap.ascan.scan(target_url, scanpolicyname='Light')

print("active scan id: {0}".format(active_scan_id))

#now we can start monitoring the spider's status
while int(zap.ascan.status(active_scan_id)) < 100:
    print("Current Status of ZAP Active Scan: {0}%".format(zap.ascan.status(active_scan_id)))
    time.sleep(10)

active scan id: 0
Current Status of ZAP Active Scan: 0%
Current Status of ZAP Active Scan: 89%
Current Status of ZAP Active Scan: 99%


### Shutdown ZAP

In [20]:
zap.core.shutdown()

'OK'

### Delete "Tainted" Session files in `temp_session`

In [21]:
temp_dir = 'temp_session'
for filename in glob.glob(os.path.join(temp_dir, '*.*')):
    os.remove(filename)